In [4]:
import pandas as pd
import sys
import itertools
import numpy as np
import random
from pandarallel import pandarallel
import requests
from tqdm import tqdm
import glob
import json
import httplib2

h = httplib2.Http()

In [ ]:
snps_23me = list(
    pd.read_csv("23andme_snps_2018.txt", header=3, delimiter="\t")['snp'])
df_map = pd.read_csv(
    "hapmap3_r2_b36_fwd.qc.poly/CEU/hapmap3_r3_b36_fwd.CEU.qc.poly.map", sep="\t", header=None)
df_map['index'] = range(0, len(df_map))


In [3]:
df_map

,0,1,2,3,index
0,1,rs6650104,0,554340,0
1,1,rs11510103,0,557616,1
2,1,rs12565286,0,711153,2
3,1,rs3094315,0,742429,3
4,1,rs3131972,0,742584,4
...,...,...,...,...,...
1397809,25,rs1973881,0,154883690,1397809
1397810,26,rs3021088,0,5461,1397810
1397811,26,rs2854139,0,9819,1397811
1397812,26,rs3915952,0,11252,1397812


In [4]:
to_exclude = list(df_map[~df_map[1].isin(snps_23me)]['index'])
to_exclude_ped = [(t*2,t*2+1) for t in to_exclude]
to_exclude_ped = list(np.array(list(itertools.chain(*to_exclude_ped)))+6)


In [7]:
df_ped = pd.read_csv(
    "hapmap3_r2_b36_fwd.qc.poly/CEU/hapmap3_r3_b36_fwd.CEU.qc.poly_tmp.ped", sep=None, header=None)


/Users/jerrybonnell/mambaforge/envs/blueberry/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [8]:
df_ped

,0,1,2,3,4,5,6,7,8,9,...,2795624,2795625,2795626,2795627,2795628,2795629,2795630,2795631,2795632,2795633
0,1328,NA06989,0,0,2,-9,0,0,A,A,...,A,A,G,G,C,C,A,A,C,C
1,1377,NA11891,0,0,1,-9,A,A,A,A,...,A,A,G,G,C,C,A,A,C,C
2,1349,NA11843,0,0,1,-9,A,A,A,A,...,A,A,G,G,C,C,G,G,C,C
3,1330,NA12341,0,0,2,-9,A,A,A,A,...,A,A,A,A,C,C,A,A,C,C
4,1444,NA12739,NA12748,NA12749,1,-9,A,A,A,A,...,A,A,G,G,C,C,A,A,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1447,NA12752,NA12760,NA12761,1,-9,A,A,A,A,...,G,A,G,G,C,C,A,A,C,C
161,1346,NA12043,0,0,1,-9,A,A,A,A,...,A,A,G,G,C,C,G,G,C,C
162,1375,NA12264,0,0,1,-9,A,A,A,A,...,G,G,G,G,C,C,A,A,C,C
163,1349,NA10854,NA11839,NA11840,2,-9,A,A,A,A,...,A,A,G,G,C,C,A,A,C,C


In [11]:
len(df_ped.columns)  # 7935308


7935308

In [20]:
df_ped.drop(df_ped.columns[to_exclude_ped], axis=1).to_csv("a.csv", header=False, index=False)
#df_ped_filtered


In [30]:
(len(df_ped.columns) - 6) // 2

3967651

#### new

In [2]:
df_ped_orig = pd.read_csv(
    "hapmap3_r2_b36_fwd.qc.poly/CEU/hapmap3_r3_b36_fwd.CEU.qc.poly_tmp_filtered.ped", 
    sep=None, header=None)

<ipython-input-2-3a538525d6ef>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df_ped_orig = pd.read_csv(


In [3]:
df_ped_orig


,0,1,2,3,4,5,6,7,8,9,...,1513242,1513243,1513244,1513245,1513246,1513247,1513248,1513249,1513250,1513251
0,1328,NA06989,0,0,2,-9,G,A,A,G,...,G,G,G,G,C,C,C,C,C,C
1,1377,NA11891,0,0,1,-9,G,A,A,G,...,T,T,A,A,C,C,C,C,C,C
2,1349,NA11843,0,0,1,-9,A,A,G,G,...,G,T,G,A,A,C,C,C,C,C
3,1330,NA12341,0,0,2,-9,G,A,A,G,...,G,G,G,G,C,C,C,C,C,C
4,1444,NA12739,NA12748,NA12749,1,-9,A,A,G,G,...,G,G,G,G,C,C,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1447,NA12752,NA12760,NA12761,1,-9,A,A,G,G,...,T,T,A,A,A,C,C,C,C,C
161,1346,NA12043,0,0,1,-9,A,A,G,G,...,T,T,A,A,A,C,C,C,C,C
162,1375,NA12264,0,0,1,-9,A,A,G,G,...,T,T,A,A,C,C,C,C,C,C
163,1349,NA10854,NA11839,NA11840,2,-9,A,A,G,G,...,0,0,G,G,0,0,C,C,C,C


In [4]:
# TODO handle missing values
df_ped = df_ped_orig.replace({"0":np.nan, 0:np.nan})

In [5]:
df_ped


,0,1,2,3,4,5,6,7,8,9,...,1513242,1513243,1513244,1513245,1513246,1513247,1513248,1513249,1513250,1513251
0,1328,NA06989,NaN,NaN,2,-9,G,A,A,G,...,G,G,G,G,C,C,C,C,C,C
1,1377,NA11891,NaN,NaN,1,-9,G,A,A,G,...,T,T,A,A,C,C,C,C,C,C
2,1349,NA11843,NaN,NaN,1,-9,A,A,G,G,...,G,T,G,A,A,C,C,C,C,C
3,1330,NA12341,NaN,NaN,2,-9,G,A,A,G,...,G,G,G,G,C,C,C,C,C,C
4,1444,NA12739,NA12748,NA12749,1,-9,A,A,G,G,...,G,G,G,G,C,C,C,C,C,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,1447,NA12752,NA12760,NA12761,1,-9,A,A,G,G,...,T,T,A,A,A,C,C,C,C,C
161,1346,NA12043,NaN,NaN,1,-9,A,A,G,G,...,T,T,A,A,A,C,C,C,C,C
162,1375,NA12264,NaN,NaN,1,-9,A,A,G,G,...,T,T,A,A,C,C,C,C,C,C
163,1349,NA10854,NA11839,NA11840,2,-9,A,A,G,G,...,NaN,NaN,G,G,NaN,NaN,C,C,C,C


In [6]:
df_map = pd.read_csv(
    "hapmap3_r2_b36_fwd.qc.poly/CEU/hapmap3_r3_b36_fwd.CEU.qc.poly_tmp_filtered.map",
    sep="\t", header=None)
df_map['index'] = range(0, len(df_map))


In [7]:
df_map
# SNPs  2*756908


,0,1,2,3,index
0,1,rs3094315,0,742429,0
1,1,rs3131972,0,742584,1
2,1,rs12124819,0,766409,2
3,1,rs11240777,0,788822,3
4,1,rs4970383,0,828418,4
...,...,...,...,...,...
756618,25,rs5983854,0,154871186,756618
756619,25,rs2205598,0,154871731,756619
756620,25,rs3093463,0,154881766,756620
756621,26,rs2854139,0,9819,756621


In [7]:
# rs3094315
def query_variant_info(rsid_list):
    # http://myvariant.info/v1/query/
    #url = f'https://myvariant.info/v1/variant/{rsid}?fields=_id'
    rsid_str = ",".join(rsid_list)
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    params = f'q={rsid_str}&scopes=all&fields=_id'
    res, con = h.request('http://myvariant.info/v1/query',
                        'POST', params, headers=headers)
    assert res.status == 200, print(res)
    response_json = json.loads(con)
    print(len(response_json))
    return response_json


In [8]:
query_variant_info(["rs3748597"])


3


[{'query': 'rs3748597', '_id': 'chr1:g.888659G>A', '_score': 20.984804},
 {'query': 'rs3748597', '_id': 'chr1:g.888659T>C', '_score': 8.06275},
 {'query': 'rs3748597', '_id': 'chr1:g.888659T>A', '_score': 7.814102}]

In [61]:
# json_batch_list = []
# steps = list(range(0, len(df_map), 10000))+[len(df_map)]
# for i in range(len(steps)):
#     if i==0: continue
#     query_variant_info(list(df_map[1])[steps[i-1]:steps[i]])


In [8]:
with open("variantinfo/all.json", "r") as f:
    all_json = json.load(f)
assert len(all_json.keys()) >= len(df_map)


In [9]:
def variant_info_from_json(rsid):
    rsid_content = all_json[rsid]
    if False in rsid_content:
        return np.nan, np.nan
    # extract that G>T and take the T as mutation and G as reference
    ref2alt = [(entry[entry.index(">")-1], entry[entry.index(">")+1]) 
            for entry in rsid_content if ">" in entry]
    if len(ref2alt) == 0:
        return -1, np.nan
    # while mutations could be different the reference should be the same
    # across the board
    if len(set([r[0] for r in ref2alt])) == 1:
        return ref2alt[0][0], list(set([r[1] for r in ref2alt]))
    else: 
        return -2, np.nan


In [11]:
for rs in list(df_map[1]):
    variant_info_from_json(rs)

In [10]:
variant_info_from_json("rs3748597")


(-2, nan)

In [12]:
tqdm.pandas()
df_map['ref'], df_map['alt'] = zip(
    *df_map[1].progress_apply(variant_info_from_json))


100%|██████████| 756623/756623 [00:02<00:00, 347479.03it/s]
/home/graph/jbonnell/miniconda3/envs/blueberry/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [13]:
#df_map[df_map['ref'] == -1] # TODO rsid's where there isn't >  (2)
#df_map[df_map['ref'] == -2] # TODO rsid's where the ref is different among possibilities (190)
#df_map[df_map['ref'] != df_map['ref']] # TODO rsid's that could not be found in db (94)
df_map

,0,1,2,3,index,ref,alt
0,1,rs3094315,0,742429,0,G,"[A, T]"
1,1,rs3131972,0,742584,1,A,"[G, C, T]"
2,1,rs12124819,0,766409,2,A,[G]
3,1,rs11240777,0,788822,3,G,[A]
4,1,rs4970383,0,828418,4,C,"[A, G]"
...,...,...,...,...,...,...,...
756618,25,rs5983854,0,154871186,756618,C,"[A, T]"
756619,25,rs2205598,0,154871731,756619,C,[T]
756620,25,rs3093463,0,154881766,756620,C,[A]
756621,26,rs2854139,0,9819,756621,C,[T]


In [11]:
df_map.to_csv("map_filtered_myvariant.csv")

In [112]:
# pd.concat([df_map[df_map['ref'] == -1],
#            df_map[df_map['ref'] == -2],
#            df_map[df_map['ref'] != df_map['ref']]])[1]\
# .to_csv("bad_snp_list.txt",index=False,header=False)

In [5]:
worksheet = pd.read_excel("worksheets/variantStats-v2.xlsb.xlsx", sheet_name="all").\
    drop_duplicates(subset=['SNPS'], ignore_index=True)

In [16]:
worksheet


,SNPS,REF,ALT,RAF,OR,condition
0,rs12188300,A,T,0.080310,NaN,psoriasis
1,rs13437088,C,T,0.261550,NaN,psoriasis
2,rs4379175,G,T,0.331190,NaN,psoriasis
3,rs17716942,T,C,0.143274,NaN,psoriasis
4,rs6677595,C,T,0.625855,NaN,psoriasis
5,rs12720356,A,C,0.086250,NaN,psoriasis
6,rs582757,C,T,0.724810,NaN,psoriasis
7,rs7552167,A,G,0.860656,NaN,psoriasis
8,rs1295685,A,G,0.804901,NaN,psoriasis
9,rs892085,G,A,0.556606,NaN,psoriasis


In [17]:
np.random.seed(2022)
df_ped_bootstrap = pd.concat(
    [df_ped, df_ped.sample(n=len(df_ped),replace=True), df_ped.sample(n=len(df_ped),replace=True)]) 

In [18]:
#df_ped_bootstrap # 165 * 3 rows 
from sklearn.model_selection import train_test_split
#np.random.seed(2022)
df_cases, df_control = train_test_split(df_ped_bootstrap, test_size=0.5)
df_cases['group'] = "cases"
df_control['group'] = "control"
df_cases = df_cases.reset_index(drop=True)
df_control = df_control.reset_index(drop=True)
# 247 + 248 = 495


<ipython-input-18-d12e1eed16b0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cases['group'] = "cases"
<ipython-input-18-d12e1eed16b0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_control['group'] = "control"


In [19]:
worksheet.merge(df_map, left_on="SNPS", right_on=1, how='inner')


,SNPS,REF,ALT,RAF,OR,condition,0,1,2,3,index,ref,alt
0,rs12188300,A,T,0.080310,NaN,psoriasis,5,rs12188300,0,158762105,256739,A,"[G, T]"
1,rs13437088,C,T,0.261550,NaN,psoriasis,6,rs13437088,0,31463098,276974,C,[T]
2,rs4379175,G,T,0.331190,NaN,psoriasis,5,rs4379175,0,158737506,256730,G,[T]
3,rs17716942,T,C,0.143274,NaN,psoriasis,2,rs17716942,0,162968937,101411,T,[C]
4,rs6677595,C,T,0.625855,NaN,psoriasis,1,rs6677595,0,150856811,33177,C,"[A, G, T]"
5,rs12720356,A,C,0.086250,NaN,psoriasis,19,rs12720356,0,10330975,692157,A,"[G, C]"
6,rs582757,C,T,0.724810,NaN,psoriasis,6,rs582757,0,138239517,304583,C,"[A, T]"
7,rs7552167,A,G,0.860656,NaN,psoriasis,1,rs7552167,0,24391230,7166,A,"[G, T]"
8,rs1295685,A,G,0.804901,NaN,psoriasis,5,rs1295685,0,132024344,249416,A,"[G, T]"
9,rs892085,G,A,0.556606,NaN,psoriasis,19,rs892085,0,10679092,692240,G,[A]


In [20]:
# NOTE only picking variants that are consistent b/w spreadsheet and db
#signature = worksheet.merge(df_map, left_on="SNPS", right_on=1, how='inner').iloc[[0,1,4,7,8,9,10,11,14,17],:]#.sample(n=12,replace=False)
wksht_map = worksheet.merge(df_map, left_on="SNPS", right_on=1, how='inner')
wksht_map = wksht_map.drop(wksht_map.index[[40]]).reset_index(drop=True)
wksht_map['orig_index'] = wksht_map['index']
wksht_map['index'] = 2*wksht_map['index']+6
wksht_map
# 12 * 24 checks


,SNPS,REF,ALT,RAF,OR,condition,0,1,2,3,index,ref,alt,orig_index
0,rs12188300,A,T,0.080310,NaN,psoriasis,5,rs12188300,0,158762105,513484,A,"[G, T]",256739
1,rs13437088,C,T,0.261550,NaN,psoriasis,6,rs13437088,0,31463098,553954,C,[T],276974
2,rs4379175,G,T,0.331190,NaN,psoriasis,5,rs4379175,0,158737506,513466,G,[T],256730
3,rs17716942,T,C,0.143274,NaN,psoriasis,2,rs17716942,0,162968937,202828,T,[C],101411
4,rs6677595,C,T,0.625855,NaN,psoriasis,1,rs6677595,0,150856811,66360,C,"[A, G, T]",33177
5,rs12720356,A,C,0.086250,NaN,psoriasis,19,rs12720356,0,10330975,1384320,A,"[G, C]",692157
6,rs582757,C,T,0.724810,NaN,psoriasis,6,rs582757,0,138239517,609172,C,"[A, T]",304583
7,rs7552167,A,G,0.860656,NaN,psoriasis,1,rs7552167,0,24391230,14338,A,"[G, T]",7166
8,rs1295685,A,G,0.804901,NaN,psoriasis,5,rs1295685,0,132024344,498838,A,"[G, T]",249416
9,rs892085,G,A,0.556606,NaN,psoriasis,19,rs892085,0,10679092,1384486,G,[A],692240


In [21]:
num_blocks = 10
block_size = len(df_cases) // num_blocks
blocks = np.arange(0, len(df_cases), block_size)
blocks[-1] = len(df_cases) # make the last one go all the way
blocks = np.diff(blocks)
blocks


array([24, 24, 24, 24, 24, 24, 24, 24, 24, 31])

In [22]:
#np.random.seed(2022)
df_cases_blocks = []
cases_index_bag = list(range(len(df_cases)))
for block in tqdm(blocks): 
    indices = np.random.choice(cases_index_bag, size=block, replace=False)
    df_cases_blocks.append(df_cases.iloc[indices, :].copy(deep=True))
    # remove from the bag
    for selected in indices:
        cases_index_bag.remove(selected)
# bag should be empty when this is done 
assert len(cases_index_bag) == 0
# parts should sum to the total 
assert sum([len(df) for df in df_cases_blocks]) == len(df_cases)

with open("gen_info/blocks.info", "w") as f:
    for index, block in enumerate(df_cases_blocks):
        f.write(f"----block {index}\n")
        f.write(f"{list(block.index)}\n")
df_cases_blocks = [df.reset_index(drop=True) for df in df_cases_blocks]

100%|██████████| 10/10 [00:08<00:00,  1.16it/s]


In [23]:
print([len(b) for b in df_cases_blocks])
sum([len(b) for b in df_cases_blocks])


[24, 24, 24, 24, 24, 24, 24, 24, 24, 31]


247

In [24]:
# each block is associated with a different signature 
#np.random.seed(2022)
sig_list = []
rsid_contribution = 10
for block_num in list(range(num_blocks)):
    sig_list.append(wksht_map.sample(n=rsid_contribution, replace=False))

with open("gen_info/blocks.info", "a") as f:
    for index,sig in enumerate(sig_list):
        f.write(f"----block {index}\n")
        f.write(f"{list(sig['SNPS'])}\n")
sig_list = [df.reset_index(drop=True) for df in sig_list]
len(sig_list)

10

In [25]:
sig_list[1]

,SNPS,REF,ALT,RAF,OR,condition,0,1,2,3,index,ref,alt,orig_index
0,rs12445568,T,C,0.363960,NaN,psoriasis,16,rs12445568,0,30912313,1262960,T,[C],631477
1,rs2041733,T,C,0.549185,NaN,atopic dermtitis,16,rs2041733,0,11137090,1253216,T,"[G, C]",626605
2,rs1250546,A,G,0.416120,NaN,psoriasis,10,rs1250546,0,80702538,909928,A,"[G, T]",454961
3,rs12295535,C,T,0.028270,NaN,atopic dermtitis,11,rs12295535,0,36388600,966478,C,[T],483236
4,rs12153855,T,C,0.105514,NaN,atopic dermtitis,6,rs12153855,0,32182782,555424,T,[C],277709
5,rs582757,C,T,0.724810,NaN,psoriasis,6,rs582757,0,138239517,609172,C,"[A, T]",304583
6,rs6677595,C,T,0.625855,NaN,psoriasis,1,rs6677595,0,150856811,66360,C,"[A, G, T]",33177
7,rs176095,A,G,0.219141,NaN,atopic dermtitis,6,rs176095,0,32266297,555566,A,[G],277780
8,rs10738626,C,T,0.480000,NaN,atopic dermtitis,9,rs10738626,0,22363457,812094,C,"[A, G, T]",406044
9,rs12720356,A,C,0.086250,NaN,psoriasis,19,rs12720356,0,10330975,1384320,A,"[G, C]",692157


In [26]:
from collections import defaultdict
def flip(p):
    return True if random.random() < p else False


In [27]:
sum([len(b) for b in df_cases_blocks])
block_lens = [len(b) for b in df_cases_blocks]
block_lens


[24, 24, 24, 24, 24, 24, 24, 24, 24, 31]

In [28]:
#df_cases_candidates
ests = defaultdict(list)
#np.random.seed(2022)
assert len(df_cases_blocks) == len(sig_list)
# visit each block
for b_ind in tqdm(range(len(df_cases_blocks)),total=len(df_cases_blocks)):
    # visit the signature associated with that block
    for snp in zip(sig_list[b_ind]['ALT'], sig_list[b_ind]['RAF'], 
                   sig_list[b_ind]['index'], sig_list[b_ind]['SNPS']):
        # simulate introduction of variant for each individual
        # in that block according to RAF 
        num_het_edits = 0
        num_hom_edits = 0
        assert len(df_cases_blocks[b_ind]) == block_lens[b_ind]
        for i in range(len(df_cases_blocks[b_ind])):
            if flip(snp[1]):
                df_cases_blocks[b_ind].at[i, snp[2]] = snp[0]
                num_het_edits += 1
                # no more than half the chance of RAF to introduce
                # a homozygous variant 
                if flip(snp[1]/2):
                    df_cases_blocks[b_ind].at[i, snp[2]+1] = snp[0]
                    num_hom_edits += 1
        assert len(df_cases_blocks[b_ind]) == block_lens[b_ind]
        ests[snp[-1]].append((num_het_edits / len(df_cases_blocks[b_ind]), 
                    snp[1], 
                    num_hom_edits / len(df_cases_blocks[b_ind]),
                    snp[1]/2))

100%|██████████| 10/10 [00:00<00:00, 56.38it/s]


In [35]:
#ests


In [29]:
df_cases_mutated = pd.concat(df_cases_blocks).reset_index(drop=True)
df_cases_mutated


,0,1,2,3,4,5,6,7,8,9,...,1513243,1513244,1513245,1513246,1513247,1513248,1513249,1513250,1513251,group
0,1349,NA10853,NA11843,NaN,1,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
1,1423,NA10843,NA11919,NA11920,2,-9,A,A,G,G,...,T,A,A,A,C,C,C,C,C,cases
2,1330,NA12336,NA12342,NA12343,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
3,1341,NA06993,NaN,NaN,1,-9,G,A,A,G,...,T,G,A,C,C,C,C,C,C,cases
4,1459,NA12875,NaN,NaN,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,1334,NA12239,NaN,NaN,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
243,1420,NA10839,NA12005,NA12006,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
244,1330,NA12341,NaN,NaN,2,-9,G,A,A,G,...,G,G,G,C,C,C,C,C,C,cases
245,1447,NA12762,NaN,NaN,1,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases


In [30]:
series_list = []
for i in tqdm(range(6, 1513251+1), total=len(range(6, 1513251+1))):
    z = list(zip(list(df_cases_mutated[i].value_counts().index),
                 list(df_cases_mutated[i].value_counts().values)))
    z = [list(i[0]*i[1]) for i in z]
    series_list.append(pd.Series([item for sublist in z for item in sublist]))
a = pd.concat(series_list)

100%|██████████| 1513246/1513246 [16:08<00:00, 1562.38it/s]


In [31]:
series_list = []
for i in tqdm(range(6, 1513251+1), total=len(range(6, 1513251+1))):
    z = list(zip(list(df_cases[i].value_counts().index), 
                 list(df_cases[i].value_counts().values)))
    z = [list(i[0]*i[1]) for i in z]
    series_list.append(pd.Series([item for sublist in z for item in sublist]))
b = pd.concat(series_list)

100%|██████████| 1513246/1513246 [16:28<00:00, 1531.51it/s] 


In [33]:
a.value_counts()-b.value_counts()

C     11
G    200
T      0
A   -208
dtype: int64

In [36]:
del a 
del b

In [37]:
df_together = pd.concat([df_cases_mutated, df_control]).reset_index(drop=True)
# df_together = df_together[['group'] +
#                  [col for col in df_together.columns if col != 'group']]
df_together
assert len(df_together) == len(df_ped_bootstrap)


In [38]:
df_together.to_hdf("df_together.h5", key='db', mode='w', index=False)

/home/graph/jbonnell/miniconda3/envs/blueberry/lib/python3.9/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)
/home/graph/jbonnell/miniconda3/envs/blueberry/lib/python3.9/site-packages/pandas/io/pytables.py:280: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_items] [items->None]

  f(store)
/home/graph/jbonnell/miniconda3/envs/blueberry/lib/python3.9/site-packages/pandas/core/generic.py:2605: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index([      1,       2,       3,       6,       7,       8,       9,      10,
            11,      12,
       ...
       1513243,

In [5]:
df = pd.read_hdf("df_together.h5")

In [7]:
df

,0,1,2,3,4,5,6,7,8,9,...,1513243,1513244,1513245,1513246,1513247,1513248,1513249,1513250,1513251,group
0,1349,NA10853,NA11843,NaN,1,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
1,1423,NA10843,NA11919,NA11920,2,-9,A,A,G,G,...,T,A,A,A,C,C,C,C,C,cases
2,1330,NA12336,NA12342,NA12343,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
3,1341,NA06993,NaN,NaN,1,-9,G,A,A,G,...,T,G,A,C,C,C,C,C,C,cases
4,1459,NA12875,NaN,NaN,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1345,NA07347,NaN,NaN,1,-9,A,A,G,G,...,T,A,A,C,C,C,C,C,C,control
491,1345,NA07348,NA07357,NA07345,2,-9,G,A,A,G,...,T,NaN,NaN,C,C,C,C,C,C,control
492,1377,NA11893,NaN,NaN,1,-9,G,A,A,G,...,T,G,A,C,C,C,C,C,C,control
493,1344,NA12057,NaN,NaN,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,control


In [9]:
df_together_recoded = df[[0,1,2,3,4,5,'group']].copy()
# df_together_recoded = df_together.iloc[:, 0].copy()

In [10]:
df_together_recoded.to_csv("df_recoded_metadata_part.csv")

In [44]:
chunk_size = 2000

#1513253
alelle_pairs = list(zip(range(6, 1513251+1, 2), range(7, 1513251+1, 2)))
alelle_pairs = [list(p) for p in alelle_pairs]
len(alelle_pairs)

a = np.append(np.arange(0, len(alelle_pairs), chunk_size),
              np.array(len(alelle_pairs)))
b = np.append(np.arange(chunk_size, len(alelle_pairs), chunk_size),
              np.array(len(alelle_pairs)))
chunks = list(zip(a, b))


In [46]:
with open("recode_chunks_new", "w") as f:
    [f.write(f"{l[0]} {l[1]}\n") for l in chunks]

In [47]:
# now parallel...

In [5]:
import os
with open("recode_chunks_new", "r") as f:
    chunks = [l.split() for l in f.readlines()]
remain = []
for chunk in chunks: 
    if not os.path.exists(f"recode_chunks/recode_chunk_{chunk[0]}_{chunk[1]}.h5"):
        remain.append(chunk)
len(remain)

181

In [ ]:
with open("recode_chunks_new2", "w") as f:
    [f.write(f"{l[0]} {l[1]}\n") for l in remain]

In [25]:
df_map['ped_index'] = 2*df_map['index']+6
df_map
# alelle_pairs = list(zip(range(6, 1513251+1, 2), range(7, 1513251+1, 2)))


,0,1,2,3,index,ref,alt,ped_index
0,1,rs3094315,0,742429,0,G,"[T, A]",6
1,1,rs3131972,0,742584,1,A,"[G, T, C]",8
2,1,rs12124819,0,766409,2,A,[G],10
3,1,rs11240777,0,788822,3,G,[A],12
4,1,rs4970383,0,828418,4,C,"[G, A]",14
...,...,...,...,...,...,...,...,...
756618,25,rs5983854,0,154871186,756618,C,"[T, A]",1513242
756619,25,rs2205598,0,154871731,756619,C,[T],1513244
756620,25,rs3093463,0,154881766,756620,C,[A],1513246
756621,26,rs2854139,0,9819,756621,C,[T],1513248


In [71]:
xxx = pd.DataFrame({'a': [1, 2, 3], 'b': [1, 2, 3],
                   'c': [1, 2, 3], 'd': [1, 2, 3], 'e': [1, 2, 3]})
xxx.iloc[:, 2:4]


,c,d
0,1,1
1,2,2
2,3,3


In [72]:
xxx.iloc[:, 2:4].apply(lambda x: x[0]+x[1], axis=1)


0    2
1    4
2    6
dtype: int64

In [26]:
def codify_nucleic_acid_pair(x, pos1, pos2):
    # keep missing data missing
    if x[pos1] != x[pos1] or x[pos2] != x[pos2]:
        return np.nan
    ref = df_map[df_map['ped_index'] == pos1]['ref'].to_list()
    alt = df_map[df_map['ped_index'] == pos1]['alt'].to_list()[0]
    # the allele has an appearance of something other than ref 
    # or alt
    if sum([allele not in ref+alt for allele in x.to_list()]) > 0:
        return 3
    mutations = sum([allele in alt for allele in x.to_list()])
    return mutations


In [33]:
df_together.iloc[:, 6:6+2]


,5,6
0,-9,A
1,-9,NaN
2,-9,A
3,-9,A
4,-9,A
...,...,...
490,-9,G
491,-9,G
492,-9,A
493,-9,A


In [37]:
df_together

,0,1,2,3,4,5,6,7,8,9,...,1513243,1513244,1513245,1513246,1513247,1513248,1513249,1513250,1513251,group
0,1345,NA07349,NA07347,NA07346,1,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
1,1330,NA12342,NaN,NaN,1,-9,A,A,G,G,...,T,A,A,C,C,C,C,C,C,cases
2,1420,NA10839,NA12005,NA12006,2,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
3,1421,NA12282,NaN,NaN,1,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,cases
4,1340,NA06994,NaN,NaN,1,-9,A,A,G,G,...,T,A,A,A,C,C,C,C,C,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,13281,NA12347,NaN,NaN,1,-9,G,G,A,A,...,T,G,A,C,C,C,C,C,C,control
491,1454,NA12801,NA12812,NA12813,1,-9,A,A,G,G,...,T,A,A,C,C,C,C,C,C,control
492,1377,NA10865,NA11891,NA11892,1,-9,A,A,G,G,...,T,G,A,C,C,C,C,C,C,control
493,1346,NA10852,NA12045,NaN,2,-9,A,A,G,G,...,G,G,G,C,C,C,C,C,C,control


In [32]:
df_together.to_hdf("df_together.h5", key='db', mode='w',index=False)


/Users/jerrybonnell/mambaforge/envs/blueberry/lib/python3.9/site-packages/pandas/io/pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis0] [items->None]

  f(store)
/Users/jerrybonnell/mambaforge/envs/blueberry/lib/python3.9/site-packages/pandas/io/pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_items] [items->None]

  f(store)
/Users/jerrybonnell/mambaforge/envs/blueberry/lib/python3.9/site-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index([      1,       2,       3,       6,       7,       8,       9,      10,
            11,      12,
       ...
       1513243, 15

In [30]:
df_together.to_csv("df_together.csv", sep=" ", header=False, index=False)


In [34]:
alelle_pairs = list(zip(range(6, 1513251+1, 2), range(7, 1513251+1, 2)))
alelle_pairs = [list(p) for p in alelle_pairs]
#alelle_pairs[:10]


In [106]:
# for pair in tqdm(alelle_pairs):
#     res = []
#     for x in df_together[pair].to_numpy():
#         res.append(codify_nucleic_acid_pair(x, pair[0], pair[1]))

In [35]:
df_together[alelle_pairs[0]].apply(
    codify_nucleic_acid_pair,
    args=(alelle_pairs[0][0], alelle_pairs[0][1]),
    axis=1)


0      2.0
1      1.0
2      2.0
3      2.0
4      1.0
      ... 
490    0.0
491    2.0
492    2.0
493    2.0
494    2.0
Length: 495, dtype: float64

In [36]:
a = []
for pair in tqdm(alelle_pairs[:5]):
    a.append(df_together[pair].apply(
        codify_nucleic_acid_pair,
        args=(pair[0], pair[1]),
        axis=1))


100%|██████████| 5/5 [00:02<00:00,  1.79it/s]


In [38]:
pd.concat(a, axis=1)

,0,1,2,3,4
0,2.0,2,1.0,0.0,1
1,1.0,1,0.0,1.0,0
2,2.0,2,2.0,0.0,0
3,2.0,2,1.0,0.0,0
4,1.0,1,0.0,1.0,1
...,...,...,...,...,...
490,0.0,0,0.0,1.0,1
491,2.0,2,1.0,0.0,0
492,2.0,2,0.0,0.0,0
493,2.0,2,0.0,1.0,0


In [64]:
for row in tqdm(zip(df_map['ped_index']), total=len(df_map)):
    df_together_recoded[row[0]] = df_together[[row[0],row[0]+1]].apply(
            codify_nucleic_acid_pair,
            args=(row[0], row[0]+1),
            axis=1)
    # df_together_recoded[row[0]] = df_together.iloc[:, row[0]:row[0]+2].apply(
    #     codify_nucleic_acid_pair,
    #     args=(row[0], row[0]+1),
    #     axis=1)

  0%|          | 13/756623 [00:07<123:27:28,  1.70it/s]


KeyboardInterrupt: 

In [63]:
(len(alelle_pairs)//1000)


756

In [72]:
a = np.append(np.arange(0, len(alelle_pairs), 1000),np.array(len(alelle_pairs)))
b = np.append(np.arange(1000, len(alelle_pairs), 1000),
          np.array(len(alelle_pairs)))
pd.DataFrame(list(zip(a,b)))


,0,1
0,0,1000
1,1000,2000
2,2000,3000
3,3000,4000
4,4000,5000
...,...,...
752,752000,753000
753,753000,754000
754,754000,755000
755,755000,756000


In [52]:
len(alelle_pairs)

756623

In [54]:
# df_together.iloc[:, df_map.iloc[0, -1]:df_map.iloc[0, -1]+2].apply(
#     codify_nucleic_acid_pair,
#     args=(df_map.iloc[0, -1], df_map.iloc[0, -1]+1), 
#     axis=1)

### post recoding

In [2]:
df_map = pd.read_csv(
    "hapmap3_r2_b36_fwd.qc.poly/CEU/hapmap3_r3_b36_fwd.CEU.qc.poly_tmp_filtered.map",
    sep="\t", header=None)
df_map['index'] = range(0, len(df_map))
df_map


,0,1,2,3,index
0,1,rs3094315,0,742429,0
1,1,rs3131972,0,742584,1
2,1,rs12124819,0,766409,2
3,1,rs11240777,0,788822,3
4,1,rs4970383,0,828418,4
...,...,...,...,...,...
756618,25,rs5983854,0,154871186,756618
756619,25,rs2205598,0,154871731,756619
756620,25,rs3093463,0,154881766,756620
756621,26,rs2854139,0,9819,756621


In [3]:
worksheet = pd.read_excel(
    "worksheets/variantStats-v2.xlsb.xlsx", sheet_name="all").\
    drop_duplicates(subset=['SNPS'])

In [4]:
worksheet[worksheet['SNPS'].isin(list(worksheet[worksheet.duplicated(subset=['SNPS'])]['SNPS']))]

,SNPS,REF,ALT,RAF,OR,condition


In [5]:
alelle_pairs = list(zip(range(6, 1513251+1, 2), range(7, 1513251+1, 2)))
alelle_pairs = [list(p) for p in alelle_pairs]
len(alelle_pairs)


756623

In [6]:
chunk_size = 2000


In [7]:
metadata_part = pd.read_csv("df_recoded_metadata_part.csv", index_col=0)
metadata_part.columns = ['FID', 'IID', 'IIDF',
                         'IIDM', 'SEX', 'PHENOTYPE', 'group']
metadata_part

,FID,IID,IIDF,IIDM,SEX,PHENOTYPE,group
0,1349,NA10853,NA11843,NaN,1,-9,cases
1,1423,NA10843,NA11919,NA11920,2,-9,cases
2,1330,NA12336,NA12342,NA12343,2,-9,cases
3,1341,NA06993,NaN,NaN,1,-9,cases
4,1459,NA12875,NaN,NaN,2,-9,cases
...,...,...,...,...,...,...,...
490,1345,NA07347,NaN,NaN,1,-9,control
491,1345,NA07348,NA07357,NA07345,2,-9,control
492,1377,NA11893,NaN,NaN,1,-9,control
493,1344,NA12057,NaN,NaN,2,-9,control


In [8]:
chunk_dfs = []
with open("recode_chunks_new", "r") as f:
    chunks = [l.split() for l in f.readlines()]
for chunk in tqdm(chunks):
    chunk_dfs.append(pd.read_hdf(f"recode_chunks/recode_chunk_{chunk[0]}_{chunk[1]}.h5"))
recoded_df = pd.concat(chunk_dfs, axis=1)
recoded_df.columns = list(range(len(df_map)))
recoded_df = pd.concat([metadata_part, recoded_df], axis=1)

100%|██████████| 379/379 [00:03<00:00, 110.79it/s]


In [9]:
recoded_df
# TODO sex distribution 50/50?
# 495 rows × 756630


,FID,IID,IIDF,IIDM,SEX,PHENOTYPE,group,0,1,2,...,756613,756614,756615,756616,756617,756618,756619,756620,756621,756622
0,1349,NA10853,NA11843,NaN,1,-9,cases,2.0,2,1.0,...,3.0,1.0,3.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0
1,1423,NA10843,NA11919,NA11920,2,-9,cases,2.0,2,1.0,...,3.0,2.0,3.0,0.0,2.0,2.0,3.0,1.0,0.0,0.0
2,1330,NA12336,NA12342,NA12343,2,-9,cases,2.0,2,2.0,...,NaN,2.0,3.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0
3,1341,NA06993,NaN,NaN,1,-9,cases,1.0,1,0.0,...,3.0,1.0,3.0,1.0,2.0,3.0,3.0,0.0,0.0,0.0
4,1459,NA12875,NaN,NaN,2,-9,cases,2.0,2,1.0,...,3.0,1.0,3.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1345,NA07347,NaN,NaN,1,-9,control,2.0,2,0.0,...,3.0,2.0,3.0,0.0,2.0,2.0,3.0,0.0,0.0,0.0
491,1345,NA07348,NA07357,NA07345,2,-9,control,1.0,1,0.0,...,3.0,1.0,3.0,1.0,3.0,3.0,NaN,0.0,0.0,0.0
492,1377,NA11893,NaN,NaN,1,-9,control,1.0,1,1.0,...,3.0,1.0,3.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0
493,1344,NA12057,NaN,NaN,2,-9,control,2.0,2,2.0,...,3.0,1.0,3.0,1.0,3.0,3.0,3.0,0.0,0.0,0.0


In [10]:
to_drop = []
for i in tqdm(range(0, 756622+1)):
    #3 in set(recoded_df[i].value_counts().index):
    if recoded_df[i].isna().sum().sum() > 0: #or \
        to_drop.append(i)

100%|██████████| 756623/756623 [01:28<00:00, 8533.64it/s]


In [11]:
len(to_drop)  # 194856


194856

In [12]:
to_drop_snps = list(df_map[df_map['index'].isin(to_drop)][1])
to_drop_index = list(df_map[df_map['index'].isin(to_drop)]['index'])

In [3]:
with open("gen_info/blocks.info", "r") as f:
    sig_info = [eval(l.strip()) for l in f.readlines()[20:] if "----" not in l]
sig_info = set([item for sublist in sig_info for item in sublist])
len(sig_info)

40

In [8]:
print(len(set(worksheet['SNPS'].tolist())))
set(worksheet['SNPS'].tolist()) - sig_info
# {'rs10994675', 'rs1327914', 'rs7130588'} 
# hapmap3 in total has 42 (worksheet total 43) snps;
# 1 of them has a discrepancy (rs10994675)
# 1 of them isnt selected in simulation (rs1327914)
# 1 of them isnt in hapmap (rs7130588)

43


{'rs10994675', 'rs1327914', 'rs7130588'}

In [14]:
sig_positions = list(df_map[df_map[1].isin(sig_info)].merge(worksheet, how='inner', left_on=1, right_on='SNPS')[['index','condition']].itertuples(index=False, name=None))
#sig_positions = list(df_map[df_map[1].isin(sig_info)])
sig_positions


[(7166, 'psoriasis'),
 (33138, 'atopic dermtitis'),
 (33177, 'psoriasis'),
 (79672, 'psoriasis'),
 (87825, 'atopic dermtitis'),
 (101395, 'psoriasis'),
 (101411, 'psoriasis'),
 (249414, 'atopic dermtitis'),
 (249416, 'psoriasis'),
 (249417, 'atopic dermtitis'),
 (249418, 'atopic dermtitis'),
 (256730, 'psoriasis'),
 (256739, 'psoriasis'),
 (263657, 'psoriasis'),
 (276974, 'psoriasis'),
 (277709, 'atopic dermtitis'),
 (277780, 'atopic dermtitis'),
 (304583, 'psoriasis'),
 (406044, 'atopic dermtitis'),
 (409078, 'psoriasis'),
 (450736, 'atopic dermtitis'),
 (454961, 'psoriasis'),
 (483236, 'atopic dermtitis'),
 (483239, 'atopic dermtitis'),
 (488489, 'psoriasis'),
 (488804, 'atopic dermtitis'),
 (491339, 'atopic dermtitis'),
 (500628, 'psoriasis'),
 (579647, 'psoriasis'),
 (609231, 'atopic dermtitis'),
 (626605, 'atopic dermtitis'),
 (631477, 'psoriasis'),
 (656155, 'atopic dermtitis'),
 (657611, 'atopic dermtitis'),
 (666103, 'psoriasis'),
 (680734, 'psoriasis'),
 (692157, 'psoriasis'),

In [15]:
with open("gen_info/sig.info", "w") as f:
    f.writelines([f"{l[0]} {l[1]}\n" for l in sig_positions])

In [16]:
to_drop_snps.index('rs582757')


74636

In [17]:
#signature[signature['SNPS'].isin(to_drop_snps)]
marked_sigs = list(sig_info.intersection(to_drop_snps))
marked_sigs
# rs582757


['rs582757', 'rs13437088', 'rs545979', 'rs4379175', 'rs11652075', 'rs10738626']

In [18]:
set(list(df_map[df_map[1].isin(marked_sigs)]['index']))


{256730, 276974, 304583, 406044, 666103, 680734}

In [19]:
# don't drop anything that was part of a signature
#to_drop = list(set(to_drop) - set(list(signature[signature['SNPS'].isin(to_drop_snps)]['index'])))
to_drop = list(set(to_drop) - set(list(df_map[df_map[1].isin(marked_sigs)]['index'])))


In [20]:
len(to_drop)  # 194856


194850

In [21]:
recoded_df = recoded_df.drop(to_drop, axis=1)


In [22]:
recoded_df
# 756630 -- 561779


,FID,IID,IIDF,IIDM,SEX,PHENOTYPE,group,1,4,5,...,756590,756592,756600,756601,756602,756605,756608,756609,756610,756611
0,1349,NA10853,NA11843,NaN,1,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
1,1423,NA10843,NA11919,NA11920,2,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
2,1330,NA12336,NA12342,NA12343,2,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
3,1341,NA06993,NaN,NaN,1,-9,cases,1,1,1,...,2,3,0,3,2,2,1,3,3,3
4,1459,NA12875,NaN,NaN,2,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1345,NA07347,NaN,NaN,1,-9,control,2,1,0,...,2,3,0,3,2,2,0,3,3,3
491,1345,NA07348,NA07357,NA07345,2,-9,control,1,1,0,...,1,3,0,3,3,2,0,3,3,3
492,1377,NA11893,NaN,NaN,1,-9,control,1,0,0,...,2,3,0,3,3,2,1,3,3,3
493,1344,NA12057,NaN,NaN,2,-9,control,2,0,0,...,1,3,0,3,2,2,1,3,3,3


In [23]:
# convert column names to rsid 
old_col_names = list(recoded_df.columns)[7:]
# df map filtered down after dropping 
df_map_relevant = df_map[df_map['index'].isin(old_col_names)][['index',1]]
assert len(df_map_relevant) == len(old_col_names)
col_map = dict(zip(list(df_map_relevant['index']), list(df_map_relevant[1])))
recoded_df.rename(columns=col_map, inplace=True)

In [24]:
import pickle
with open("gen_info/col_map.pkl", "wb") as f:
    pickle.dump(col_map, f)

In [25]:
# df.rename(columns={'old_col':'new_col', 'old_col_2':'new_col_2'}, inplace=True)
recoded_df

,FID,IID,IIDF,IIDM,SEX,PHENOTYPE,group,rs3131972,rs4970383,rs4475691,...,rs311123,rs35697189,rs17149478,rs17653586,rs700447,rs707689,rs6567787,rs2103512,rs5940620,rs4893101
0,1349,NA10853,NA11843,NaN,1,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
1,1423,NA10843,NA11919,NA11920,2,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
2,1330,NA12336,NA12342,NA12343,2,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
3,1341,NA06993,NaN,NaN,1,-9,cases,1,1,1,...,2,3,0,3,2,2,1,3,3,3
4,1459,NA12875,NaN,NaN,2,-9,cases,2,0,0,...,2,3,0,3,3,2,0,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1345,NA07347,NaN,NaN,1,-9,control,2,1,0,...,2,3,0,3,2,2,0,3,3,3
491,1345,NA07348,NA07357,NA07345,2,-9,control,1,1,0,...,1,3,0,3,3,2,0,3,3,3
492,1377,NA11893,NaN,NaN,1,-9,control,1,0,0,...,2,3,0,3,3,2,1,3,3,3
493,1344,NA12057,NaN,NaN,2,-9,control,2,0,0,...,1,3,0,3,2,2,1,3,3,3


In [26]:
weird_cols = []
for col in tqdm(list(recoded_df.columns)):
    if sum(recoded_df[col].isin([3])) > 0:
        weird_cols.append(col)


100%|██████████| 561780/561780 [01:11<00:00, 7816.37it/s]


In [27]:
[w for w in weird_cols if w in sig_info]

[]

In [28]:
# weird_cols = []
# for col in tqdm(list(recoded_df.columns)):
#     if sum(recoded_df[col].isin([3])) > 0:
#         weird_cols.append(col)
print(len(recoded_df.columns))
print(len(weird_cols))
recoded_df = recoded_df.drop(columns=weird_cols)
print(len(recoded_df.columns))

561780
609
561171


In [29]:
recoded_df


,FID,IID,IIDF,IIDM,SEX,PHENOTYPE,group,rs3131972,rs4970383,rs4475691,...,rs35289520,rs35831186,rs28416357,rs4933056,rs3814296,rs2316856,rs311123,rs17149478,rs707689,rs6567787
0,1349,NA10853,NA11843,NaN,1,-9,cases,2,0,0,...,0,0,0,1,0,0,2,0,2,0
1,1423,NA10843,NA11919,NA11920,2,-9,cases,2,0,0,...,0,1,1,1,2,1,2,0,2,0
2,1330,NA12336,NA12342,NA12343,2,-9,cases,2,0,0,...,0,0,0,0,0,0,2,0,2,0
3,1341,NA06993,NaN,NaN,1,-9,cases,1,1,1,...,0,1,2,0,1,2,2,0,2,1
4,1459,NA12875,NaN,NaN,2,-9,cases,2,0,0,...,0,1,2,0,1,1,2,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1345,NA07347,NaN,NaN,1,-9,control,2,1,0,...,0,0,1,1,1,1,2,0,2,0
491,1345,NA07348,NA07357,NA07345,2,-9,control,1,1,0,...,1,1,2,0,1,1,1,0,2,0
492,1377,NA11893,NaN,NaN,1,-9,control,1,0,0,...,0,1,1,0,2,0,2,0,2,1
493,1344,NA12057,NaN,NaN,2,-9,control,2,0,0,...,0,1,2,1,2,1,1,0,2,1


In [30]:
recoded_df.to_hdf("recoded_hapmap_ped_case_control.h5", key="db", mode="w")

/home/graph/jbonnell/miniconda3/envs/blueberry/lib/python3.9/site-packages/pandas/core/generic.py:2605: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['IID', 'IIDF', 'IIDM', 'group'], dtype='object')]

  pytables.to_hdf(


In [ ]:

# TODO bootstrap the dataset two times by sampling with replacement
# TODO split this dataset into a "cases" and "control" population; 50/50 split
# TODO introduce variants into the "cases" group 
#      - randomly select 15 of the variants from the worksheet; call this a signature 
#      - randomly select 10% of the cases from the cases group as candidates for mutation 
#      - first 6 columns of ped are useless; always jump with formula 2V + 6
#      - use the map file to jump to those columns in the ped file; 
#      - introduce the variant (REF -> ALT) with the probability given in RAF
#           - can introduce the variant just to the first one of the allele pair 
# TODO preprocessing
#      - in the ped table, recode pairs of DNA columns:
#         - "0" no variant is present
#         - "1" the variant is heterozygous (only 1 of the two in the pair has it)
#         - "2" the variant is homozygous (both)
#         - if a allele pair has NaN, leave it missing
# NOTE questions:
#             (Q: should it ever be introduced to both in the pair?)
#             (Q: what if it already has ALT?)
#             (Q: what if a allele pair we are trying to modify has missing values?)
#             some RSIDs in hapmap3 could not be found in the variantinfo database 